In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/web-page-phishing-detection-dataset/dataset_phishing.csv


In [3]:
df = pd.read_csv("/kaggle/input/web-page-phishing-detection-dataset/dataset_phishing.csv")
print(df.columns)

Index(['url', 'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens',
       'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq', 'nb_underscore',
       'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma',
       'nb_semicolumn', 'nb_dollar', 'nb_space', 'nb_www', 'nb_com',
       'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url',
       'ratio_digits_host', 'punycode', 'port', 'tld_in_path',
       'tld_in_subdomain', 'abnormal_subdomain', 'nb_subdomains',
       'prefix_suffix', 'random_domain', 'shortening_service',
       'path_extension', 'nb_redirection', 'nb_external_redirection',
       'length_words_raw', 'char_repeat', 'shortest_words_raw',
       'shortest_word_host', 'shortest_word_path', 'longest_words_raw',
       'longest_word_host', 'longest_word_path', 'avg_words_raw',
       'avg_word_host', 'avg_word_path', 'phish_hints', 'domain_in_brand',
       'brand_in_subdomain', 'brand_in_path', 'suspecious_tld',
       'statistical_report', 

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
import joblib
# === Load and Prepare Data ===
df = pd.read_csv("/kaggle/input/web-page-phishing-detection-dataset/dataset_phishing.csv")

# Drop 'url' column (we assume features are already extracted)
df = df.drop(columns=["url"])

# Encode target
df['status'] = df['status'].map({'legitimate': 0, 'phishing': 1})

X = df.drop("status", axis=1)
y = df["status"]

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Define Base Models ===
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=0)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
lr = LogisticRegression(max_iter=1000)

# === Create Ensemble Model (VotingClassifier) ===
ensemble = VotingClassifier(
    estimators=[('xgb', xgb), ('rf', rf), ('lr', lr)],
    voting='soft'  # Use 'hard' for majority class, 'soft' for probabilities
)

# === Train Ensemble ===
ensemble.fit(X_train, y_train)

# === Evaluate ===
y_pred = ensemble.predict(X_test)
print(classification_report(y_test, y_pred))
# === Save model and column order ===



              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1157
           1       0.97      0.97      0.97      1129

    accuracy                           0.97      2286
   macro avg       0.97      0.97      0.97      2286
weighted avg       0.97      0.97      0.97      2286



In [6]:
!pip install tldextract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 4.6 MB/s eta 0:00:00


In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
import joblib
import string

# Load dataset
df = pd.read_csv("/kaggle/input/web-page-phishing-detection-dataset/dataset_phishing.csv")
df = df[['url', 'status']].dropna()

# Map status to binary
df = df[df['status'].isin(['phishing', 'benign'])]
df['status'] = df['status'].map({'phishing': 1, 'benign': 0})

# === Character-Level URL Encoding ===
def encode_url(url, max_len=200):
    charset = string.ascii_letters + string.digits + string.punctuation
    char_to_int = {ch: i + 1 for i, ch in enumerate(charset)}  # start from 1
    encoded = [char_to_int.get(c, 0) for c in url[:max_len]]
    if len(encoded) < max_len:
        encoded += [0] * (max_len - len(encoded))
    return encoded

X = np.array([encode_url(u) for u in df['url']])
y = df['status'].values.astype(int)

# === Train-Test Split ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Classifiers ===
rf = RandomForestClassifier(n_estimators=100, random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# === Ensemble ===
ensemble = VotingClassifier(
    estimators=[
        ('rf', rf),
        ('xgb', xgb)
    ],
    voting='hard'  # Switch to 'soft' after verifying predict_proba
)

# === Train & Predict ===
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      1143

    accuracy                           1.00      1143
   macro avg       1.00      1.00      1.00      1143
weighted avg       1.00      1.00      1.00      1143



In [14]:
joblib.dump(model, "phishing_ensemble_ml_model.pkl")
joblib.dump(X.columns.tolist(), "feature_columns.pkl")

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [18]:
dictt={
    'max_len': 200,
    'char_to_int': {ch: i + 1 for i, ch in enumerate(string.ascii_letters + string.digits + string.punctuation)}
}
joblib.dump(dictt, "feature_columns.pkl")

['feature_columns.pkl']

In [ ]:
model = joblib.load("/kaggle/working/phishing_ensemble_ml_model.pkl")
char2idx = joblib.load("/kaggle/working/url_encoding_metadata.pkl")
maxlen = 200

def encode_url_for_inference(url):
    url = url[:maxlen].ljust(maxlen)
    return [char2idx.get(char, 0) for char in url]

test_url = "http://www.crestonwood.com/router.php"
X_new = np.array([encode_url_for_inference(test_url)])
prediction = model.predict(X_new)
print("Phishing" if prediction[0] == 1 else "Benign")